In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In this problem, you will fill out helper functions to build a bode plot of any circuit you'd like--starting with RC and CR.

Outline: 
1. Choose circuit components
2. Choose circuit values
3. Calculate impedances
4. Formulate "impedance divider"
5. Calculate transfer function
6. Calculate magnitude of transfer function
7. Calculate phase of transfer function
8. Plot magnitude of transfer function
9. Plot magnitude of transfer function



**~Choose Components~**

In [ ]:
# Change the below booleans to set up your circuit. If you'd like an RC circuit, set the top value to True. 
# If you'd like a CR circuit, set the top value to False.
RC_circuit = False
CR_circuit = not RC_circuit;

**~Choose circuit values~**

In [ ]:
C = 1e-6
R = 1000
getOmegap = lambda R, C: 1/(R*C)
omegap = getOmegap(R, C) # change this value

**~Calculate impedances~**

We want to create a getImpedance function, that, given a component, calculates the impedance at a specific omega. The omega does not have to be defined immediately (i.e., we can return a higher order function if necessary).

The skeleton code has been provided below

In [ ]:
def getImpedance(component, componentVal, omega=None):
    # fill in method so that impedance value (as a function of omega, if necessary) is returned
    # should return a function if called with 2 arguments, or a value if called with three arguments.
    # hint: use built in j
    
    '''
    getImpedance("capacitor", 1e-6, 100)
    >>> 10000j
    
    getImpedance("resistor", 1e3, 100)
    >>> 1000
    
    getImpedance("capacitor", 1e-7)
    >>> Function
    
    getImpedance("resistor", 1e6)
    >>> Function
    '''
    
    def helper(omega):
        
        if (component == "capacitor"):
            return 1/(1j*omega*componentVal)
        
        elif (component == "resistor"):
            return componentVal
        
    if (omega == None): #if we don't know the omega value yet, return a HOF that waits for omega
        return helper
    else: # shortcut if we already know omega
        return helper(omega) # replace!
    

**~Formulate "impedance divider"~**

Now we want to build the impedance divider formula to set up our transfer function. Fill in the below method to calculate the voltage difference across the second component (across the C in an RC, or across the R in a CR).

In [ ]:
def divideImpedance(R, C, omega=None):
    # fill in method so that impedance is divided properly
    # should return a function if called with 2 arguments, or a two element array if called with 3 elements
    # the HOF returns a two element array
    # the two element array should be [numer, denom]
    
    
    '''
    f = divideImpedance(1e3, 1e-6)
    output = f(1000)
    output[0]/output[1]
    >>> (1+1j)

    smalloutput = f(100)
    smalloutput[0]/smalloutput[1]
    >>> (1+0.1j)
    
    g = divideImpedance(1e-9, 1e6):
    g
    >>> Function
    '''
    # hint: you can use the above part to make this SUPER easy!
    
    def helper(omega):
        
        if RC_circuit:
            # do something
            numer = getImpedance("resistor", R)
            denom = lambda: (getImpedance("resistor", R, omega) + getImpedance("capacitor", C, omega))
            return [numer, denom]
        else: 
            # do something else
            numer = getImpedance("capacitor", C)
            denom = lambda: (getImpedance("resistor", R, omega) + getImpedance("capacitor", C, omega))
            return[numer, denom]
    
    if (omega == None):
        return helper
    else:
        return helper(omega)



**~Calculate transfer function~**

Great! We have made good progress. Now we want to formulate the Impedance divider as a transfer function with a top and bottom. This is a structural step but it will make everything later much easier!

Later on, we will curry getTransferFunc with omega to plot the transfer function.

** I have filled this out because I think the logic here is confusing. **

In [ ]:
def getTransferFunc(R, C, RC_circuit):
    if RC_circuit:
        return divideImpedance(R, C) # returns a function
    else: 
        return divideImpedance(C, R) # returns a function
    

**~Calculate magnitude of transfer function~**

Now, we will use the distance formula equation to calculate the magnitude of a transfer function at any given omega.

The getMag function takes in a (possibly complex) numerator and denominator, as well as an omega and returns a real magnitude.

In [ ]:
def getMag(transferEq, omega):
    
    num = transferEq(omega)[0](omega)
    den = transferEq(omega)[1]()
    
    #print(den)
    return np.linalg.norm(num)/np.linalg.norm(den); # change this line

**Create plotting function**

In [ ]:
def plotter(type="mag"):
    '''
    1. Figure out omegas we want to plot
    2. Create transfer function
    3. Find transfer function magnitude OR phase for several values around cutoff frequency
    4. Plot!
    '''
    
    # 1. 
    xVals = [getOmegap(R, C)/(10**(i/10)) for i in range(-50, 50)]

    # 2. 
    Rimp = getImpedance("resistor", R) # returns a function of omega
    Cimp = getImpedance("capacitor", C) # returns a function of omega
    
    f = getTransferFunc(R, C, RC_circuit) # returns a function of omega
    
    
    if(type == "mag"): # if we are plotting mag
        yVals = [getMag(f, i) for i in xVals] # to fill in later
        plt.loglog(xVals, yVals)
        plt.loglog(xVals[len(yVals)//2], yVals[len(yVals)//2], 'ro')
        plt.loglog([xVals[len(yVals)//2] for i in range(len(yVals))], yVals, 'g')
    
    else: # if we are plotting phase
        yVals = [getPhase(f, i) for i in xVals] # to fill in later
        plt.plot(xVals, yVals)
        plt.plot(xVals[len(yVals)//2], yVals[len(yVals)//2], 'ro')
        plt.plot([xVals[len(yVals)//2] for i in range(len(yVals))], yVals, 'g')
        plt.xscale("log")

~Calculate phase of transfer function~

Finally, we will use numpy's arctan2 to calculate the magnitude of a transfer function at any given omega.

The getPhase function takes in a (possibly complex) numerator and denominator, as well as an omega and returns a phase.

In [ ]:
def getPhase(transferEq, omega):
    # hint: use np.angle to calculate the arctangent of a complex number
    num = transferEq(omega)[0](omega)
    den = transferEq(omega)[1]()
    
    return np.angle(num, deg = True) - np.angle(den, deg = True)
    

**~Choose R and C values and circuit layout~**

In [ ]:
R = 1e3
C = 3.9e-6
RC_circuit = False

**~Plot magnitude and phase of transfer function~**

In [ ]:
plotter("mag")

In [ ]:
plotter("phase")